# import libraries

In [ ]:
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import random

import warnings
warnings.filterwarnings("ignore")


# perform spectral clustering

In [ ]:
A = np.zeros((1495, 1495),dtype=int)

file_name = 'friends.txt'
with open(file_name,'r') as file:
    for line in file:
        node1, node2 = map(int, line.strip().split(','))
        A[node1 - 1][node2 - 1] = 1

L = np.diag(np.sum(A, axis=1)) - A

eigenvalues, eigenvectors = np.linalg.eig(L)
sorted_indices = np.argsort(eigenvalues)
sorted_eigenvalues = eigenvalues[sorted_indices]
sorted_eigenvectors = eigenvectors[:, sorted_indices]

print(np.round(sorted_eigenvalues[:12],3))

for i in range(6):
    v = sorted_eigenvectors[:,i]
    threshold = np.max(np.abs(v)) * 0.9 # dynamically set threshold base on the values of the eigenvector
    nodes = np.where(np.abs(v) > threshold)
    print("number of nodes in cluster", i+1, ":", len(nodes[0]))


## interpret the output
- There are 6 zero eigenvalues, which means the graph has 6 connected components
- For each cluster, we can find the number of nodes within that cluster

\newpage

# cluster nodes with certain constrains
1. 150 < node_size < 750
2. cluster the data into at least 3 groups
3. the conductance between groups must smaller than 0.1
4. The conductance of $S\subset V$ is defined as
$$
conductance = \frac{E}{D}
$$

where E = number of boundary edges of S, D = total degree of S

In [ ]:
def compute_conductance(A, S):
    n = A.shape[0]
    S_complement = np.setdiff1d(np.arange(n), S) # find the complement of S
    boundary_edges = np.sum(A[np.ix_(S, S_complement)]) # find the number of boundary edges
    degree_S = np.sum(np.sum(A[S, :], axis=1)) # find the degree of S

    if degree_S == 0:
        return 1
    return boundary_edges / degree_S


def find_low_conductance_sets(A, min_size=150, max_size=750, max_conductance=0.1):

    # we need to try the proper combination of eigenvectors to find the best partition
    # try different number of clusters are also important to avoid some clusters are too small or too large

    selected_eigenvectors = sorted_eigenvectors[:, 6:15] 
    kmeans = KMeans(n_clusters=50, random_state=0)  
    labels = kmeans.fit_predict(selected_eigenvectors)
    

    sets = []
    for i in range(np.max(labels) + 1):
        S = np.where(labels == i)[0]
        if min_size <= len(S) <= max_size:
            conductance = compute_conductance(A, S)
            if conductance <= max_conductance:
                sets.append(S)

    return sets

sets = find_low_conductance_sets(A)

for i, S in enumerate(sets):
    print(f"S{i + 1}:")
    print(f"Size: {len(S)}")
    print(f"first 10 members: {S[:10]}")
    print(f"conductance: {compute_conductance(A, S)}")
    print()
